<h1>Segmenting and Clustering Chinese Restaurants in Toronto<h1/>

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem 

Cluster Toronto restaurant areas with respect to the average cost of meals in a restaurant.


## Data 

To start requesting restaurant information from foursquare website, Toronto needs to be splitted into circular areas with a radius of 300 meters, so our neighborhood centers will be 600 meters apart. A shape of Toronto in general can be descrived as trapezoid, there most/left right longitide-lattitude needs to be determined as well general width and bredth of the city.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in meters).


<table align="center">
    <tr>
        <th>         </th>
        <th>Lattitude</th>
        <th>Longitude</th>
    </tr>
    <tr>
        <th>Top right coordinates</th>
        <th>43.857698</th>
        <th>-79.169111</th>
    </tr>
    <tr>
        <th>Top left coordinates</th>
        <th>43.750724</th>
        <th>-79.640452</th>
    </tr>
    <tr>
        <th>Bottom left coordinates</th>
        <th>43.577857</th>
        <th>-79.541984</th>
    </tr>
    <tr>
        <th>Bottom right coordinates:</th>
        <th> 43.749704</th>
        <th>-79.111260</th>
    </tr>
</table>

In [3]:
from capstone import show_toronto_map

topright =      (43.857698, -79.169111)
topleft =       (43.750724, -79.640452)
bottomleft =    (43.557857, -79.541984)
bottomrigth =   (43.749704, -79.111260)

toronto_map = capstone.show_toronto_map(topright, 
                                        topleft, 
                                        bottomleft, 
                                        bottomrigth)
toronto_map

NameError: name 'capstone' is not defined

what is a reasonable radius 

In [445]:
import pyproj

def lonlat_to_xy(lat, lon):
    proj = pyproj.Proj("+proj=utm +zone=17N, +north +ellps=WGS84 +datum=WGS84 +units=m +no_defs")
    xy = proj(lon, lat)
    return xy

def xy_to_lonlat(x, y):
    proj = pyproj.Proj("+proj=utm +zone=17N, +north +ellps=WGS84 +datum=WGS84 +units=m +no_defs")
    lon, lat = proj(x, y, inverse=True)
    return lat, lon

x_1, y_1 = lonlat_to_xy(*topleft)
x_2, y_2 = lonlat_to_xy(*topright)
x_3, y_3 = lonlat_to_xy(*bottomrigth)
x_4, y_4 = lonlat_to_xy(*bottomleft)

max_horizontal_distance = np.sqrt((y_1-y_2)**2 + (x_1 - x_2)**2)
max_vertical_distance = np.sqrt((y_1-y_3)**2 + (x_1 - x_3)**2)

top_horizontal_gradient = np.divide( y_2 - y_1, x_2 - x_1)
top_horizontal_bias = y_1 - top_horizontal_gradient*x_1 

vertical_line_gradient = -top_horizontal_gradient
vertical_line_bias = y_1 + top_horizontal_gradient*x_1 

#create top line coordinates 
circle_diameter = 1290
horizontal_add_x = (circle_diameter**2/(1+top_horizontal_gradient**2))**0.5
horizontal_add_y = top_horizontal_gradient*horizontal_add_x


distance__from_hor_line = np.sqrt(3)/2*circle_diameter
vertical_add_y = (distance__from_hor_line**2/(1+vertical_line_gradient**2))**0.5
vertical_add_x = vertical_line_gradient*vertical_add_y

vertical_number_of_elements = int(max_vertical_distance//(distance__from_hor_line*2))
horizontal_number_of_elements = int(max_horizontal_distance//circle_diameter)

xy_coord = []
for i in range(vertical_number_of_elements):
    if i%2==0:
        x, y = x_1-i*vertical_add_x , y_1-i*vertical_add_y
        for j in range(horizontal_number_of_elements):
            xy_coord.append((x + j*horizontal_add_x, y + j*horizontal_add_y))
    if i%2!=0:
        x, y = x_1-i*vertical_add_x+horizontal_add_x/2, y_1-i*vertical_add_y+horizontal_add_y/2
        for j in range(horizontal_number_of_elements):
            xy_coord.append((x + j*horizontal_add_x, y + j*horizontal_add_y))


# Delete all circles below bottom line
bot_line_gradient = np.divide( y_4 - y_3, x_4 - x_3)
bot_horizontal_bias = y_4 - bot_line_gradient*x_4 

xy_filtered = []
for x_map, y_map in xy_coord:
    y_line = x_map*bot_line_gradient+bot_horizontal_bias
    if y_line < y_map: 
        xy_filtered.append((x_map, y_map))

print(len(xy_filtered))

498


In [446]:
map = folium.Map(location=[43.7, -79.4], zoom_start=10)
for xy in xy_filtered:
    lat, lon = xy_to_lonlat(*xy)
    folium.Circle([lat, lon], radius=circle_diameter/2, color='blue', fill=False).add_to(map)


map

Preparing dataframe with Toronto Postal code adresses, Neighbourhoods, Lattitude and Longitude.

In [447]:
# url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
# wiki_df = pd.read_html(requests.get(url).text)[0]
# wiki_df = wiki_df[wiki_df['Borough'] != 'Not assigned']
# coord_df = pd.read_csv('geospatial-coordinates.csv')
# wiki_df = wiki_df.merge(coord_df, how='left', on='Postal Code')
# wiki_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Using Folium library to visualise postal code areas

In [448]:
# import folium

# toronto_map = folium.Map(location=[43.7, -79.4], zoom_start=11)
# toronto_map.choropleth(
#     geo_data='Toronto2.geojson',
#     fill_color='Yellow', 
#     line_color='Red',
#     fill_opacity=0.7, 
#     line_opacity=1,
#     reset=True
# )

# postcodes = folium.map.FeatureGroup()

# for lat, lng, in zip(wiki_df.Latitude, wiki_df.Longitude):
#     postcodes.add_child(
#         folium.features.CircleMarker(
#             [lat, lng],
#             radius=5, # define how big you want the circle markers to be
#             color='yellow',
#             fill=True,
#             fill_color='blue',
#             fill_opacity=0.6
#         )
#     )

# # add pop-up text to each marker on the map
# latitudes = list(wiki_df.Latitude)
# longitudes = list(wiki_df.Longitude)
# labels = list(wiki_df['Postal Code'])

# for lat, lng, label in zip(latitudes, longitudes, labels):
#     folium.Marker([lat, lng], popup=label).add_to(toronto_map)    
    
# # add incidents to map
# toronto_map.add_child(postcodes)

Using Foursquare API to collect information about chinese restaurants in every area 

In [449]:
# CLIENT_ID = 'ULGA5LTQONIWZ5ULCDROWAYVYJ0ZOI0C35CLVMG12JJAKCFN' 
# CLIENT_SECRET = 'VMEKZXMMCBO3RWK0IFRKV5KXBABWQ1EQ3J2ACE53UAR5SVWC'
# VERSION = '20180604'
# LIMIT = 50
# search_query = 'vietnamese'# tested: chinese, asian, korean, japanese, vietnamese
# radius = 1000



function needs to be written that converts <b>result</b> request and transform it into the final desired dataframe with all the neccessary information 

Dataframe to have the following columns: <br> 

|Columns      |   
|----------   |
|Venue        |
|Venue id     | 
|Toronto area |  
|Category.name| 
|Address      |
|Longitude    |
|Lattitude    |
""

In [450]:
"""  
loop through all the types of restaurants 
"""

'  \nloop through all the types of restaurants \n'

In [451]:
# unprocessed_df = pd.DataFrame()

# toronto_postal_code = wiki_df['Postal Code'].tolist()
# toronto_borough = wiki_df['Borough'].tolist()

# list_of_df = []
# for postcode, borough in zip(toronto_postal_code, toronto_borough): 
#     near = " ".join([borough, postcode])
#     url =   f'https://api.foursquare.com/v2/venues/'\
#             f'search?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}'\
#             f'&near={near}&v={VERSION}&query={search_query}&limit={LIMIT}'

#     try: 
#         results = requests.get(url).json()
#         temp_df = pd.DataFrame(results['response']['venues'])
#         list_of_df.append(temp_df)
#     except BaseException:
#         print(f'no result for: {near}')

In [452]:
#concatinating all the dataframes and saving to csv file for future use 
# unprocessed_df = pd.concat(list_of_df, ignore_index=True)
# unprocessed_df = unprocessed_df.drop_duplicates(subset='name')
# unprocessed_df.to_csv('vietnamese_unprocessed.csv')
# unprocessed_df.shape

Processing dataframes obtained above 

In [453]:
# def unpack_name_categories(row):
#     row = eval(row)
#     if len(row) == 0:
#         return np.NaN
#     else:
#         return row[0]['name'] 

# def unpack_shortName_categories(row):
#     row = eval(row)
#     if len(row) == 0:
#         return np.NaN
#     else:
#         return row[0]['shortName'] 


# def unpack_address_location(row):
#     row = eval(row)
#     if row:
#         try:
#             return row['address']
#         except BaseException:
#             return np.NaN
#     else: 
#         return np.NaN

# def unpack_lat_location(row):
#     row = eval(row)
#     if row:
#         return row['lat']
#     else: 
#         return np.NaN

# def unpack_lng_location(row):
#     row = eval(row)
#     if row:
#         try:
#             return row['lng']
#         except BaseException:
#             return np.NaN
#     else: 
#         return np.NaN

# def unpack_postcode_location(row):
#     row = eval(row)
#     if row:
#         try:
#             return row['postalCode']
#         except BaseException:
#             return np.NaN
#     else: 
#         return np.NaN

In [454]:
# unprocessed_df = pd.read_csv('all-restaurants-unprocessed.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'all-restaurants-unprocessed.csv'

In [ ]:
# #creating a template for final results of SEARCH query 
# columns = ['VenueID', 'Venue', 'CategoryName', 'CategoryShortName', 'Address', 'PostCode', 'Longitude', 'Lattitude']
# restaurants_df = pd.DataFrame(columns=columns)

In [ ]:
# #writing a function that extract needed keys and values from categories column 
# restaurants_df.VenueID = unprocessed_df.id
# restaurants_df.Venue = unprocessed_df.name
# restaurants_df.CategoryName = unprocessed_df.apply(lambda x: unpack_name_categories(x.categories), axis=1)
# restaurants_df.CategoryShortName = unprocessed_df.apply(lambda x: unpack_shortName_categories(x.categories), axis=1)
# restaurants_df.Address = unprocessed_df.apply(lambda x: unpack_address_location(x.location), axis=1)
# restaurants_df.PostCode = unprocessed_df.apply(lambda x: unpack_postcode_location(x.location), axis=1)
# restaurants_df.Longitude = unprocessed_df.apply(lambda x: unpack_lng_location(x.location), axis=1)
# restaurants_df.Lattitude = unprocessed_df.apply(lambda x: unpack_lat_location(x.location), axis=1)

In [ ]:
# true_false = restaurants_df.CategoryShortName.value_counts() < 10
# true_false.value_counts()

short name in informative <br>
record postcode values that were searched <br>
investigate what keys location and categories column returns <br>
Category IDs corresponding to Italian restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):
<br>
split toronto into large number of circles and find restaurants from these circles <br> 

